In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# dx = 1 km; Np = 1M; Nt = 5 min
data1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
parcel1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
res='1km';t_res='5min'
Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
# res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 50M; Nz = 95
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc') #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc') #***
# res='1km'; t_res='1min_95nz'; Np_str='50e6'

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
import numpy as np
import pyfftw
import pyfftw.interfaces.numpy_fft as fftw
import matplotlib.pyplot as plt

def InvertLaplacian3DPeriodic(f, dx=1.0, dy=1.0, dz=1.0):
    nz, ny, nx = f.shape
    kx = 2 * np.pi * np.fft.fftfreq(nx, d=dx)
    ky = 2 * np.pi * np.fft.fftfreq(ny, d=dy)
    kz = 2 * np.pi * np.fft.fftfreq(nz, d=dz)
    kz, ky, kx = np.meshgrid(kz, ky, kx, indexing='ij')
    lap_k = -(kx**2 + ky**2 + kz**2)
    lap_k[0,0,0] = 1  # avoid divide-by-zero
    f_hat = fftw.fftn(f)
    phi_hat = f_hat / lap_k
    phi_hat[0,0,0] = 0  # zero mean mode
    phi = fftw.ifftn(phi_hat).real
    return phi

def LaplacianSpectral(phi, dx=1.0, dy=1.0, dz=1.0):
    nz, ny, nx = phi.shape
    kx = 2 * np.pi * np.fft.fftfreq(nx, d=dx)
    ky = 2 * np.pi * np.fft.fftfreq(ny, d=dy)
    kz = 2 * np.pi * np.fft.fftfreq(nz, d=dz)
    kz, ky, kx = np.meshgrid(kz, ky, kx, indexing='ij')
    lap_k = -(kx**2 + ky**2 + kz**2)
    phi_hat = fftw.fftn(phi)
    lap_phi_hat = lap_k * phi_hat
    lap_phi = fftw.ifftn(lap_phi_hat).real
    return lap_phi

# Grid setup
nx, ny, nz = 64, 64, 64
Lx, Ly, Lz = 16.0, 16.0, 16.0
x = np.linspace(-Lx/2, Lx/2, nx)
y = np.linspace(-Ly/2, Ly/2, ny)
z = np.linspace(-Lz/2, Lz/2, nz)
dx = x[1] - x[0]
dy = y[1] - y[0]
dz = z[1] - z[0]
Z, Y, X = np.meshgrid(z, y, x, indexing='ij')

# True phi (Gaussian)
sigma = 1.0
phi_true = np.exp(-(X**2 + Y**2 + Z**2) / (2 * sigma**2))

# Source function f = Laplacian(phi_true) analytically
f = ((X**2 + Y**2 + Z**2 - 3*sigma**2) / sigma**4) * phi_true

# Invert Laplacian to get phi
phi = InvertLaplacian3DPeriodic(f, dx=dx, dy=dy, dz=dz)

# Compute Laplacian(phi) numerically
lap_phi = LaplacianSpectral(phi, dx=dx, dy=dy, dz=dz)

# Errors
error_phi = phi - phi_true
rmse_phi = np.sqrt(np.mean(error_phi**2))
print(f"RMSE between recovered phi and true phi: {rmse_phi:.3e}")

error_f = lap_phi - f
rmse_f = np.sqrt(np.mean(error_f**2))
print(f"RMSE between Laplacian(phi) and f: {rmse_f:.3e}")

# Plot slice at middle z
slice_idx = nz // 2

plt.figure(figsize=(20,4))

plt.subplot(1,4,1)
plt.title('Source function f')
plt.imshow(f[slice_idx,:,:], origin='lower', extent=[x[0], x[-1], y[0], y[-1]])
plt.colorbar()

plt.subplot(1,4,2)
plt.title('phi')
plt.imshow(phi[slice_idx,:,:], origin='lower', extent=[x[0], x[-1], y[0], y[-1]])
plt.colorbar()

plt.subplot(1,4,3)
plt.title('Laplacian(phi)')
plt.imshow(lap_phi[slice_idx,:,:], origin='lower', extent=[x[0], x[-1], y[0], y[-1]])
plt.colorbar()

plt.subplot(1,4,4)
plt.title('Error: Lapacian(phi) - phi')
plt.imshow(error_phi[slice_idx,:,:], origin='lower', extent=[x[0], x[-1], y[0], y[-1]], cmap='bwr')
plt.colorbar()

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pyfftw
import pyfftw.interfaces.numpy_fft as fftw
import matplotlib.pyplot as plt

def InvertLaplacian3DPeriodic(f, dx=1.0, dy=1.0, dz=1.0):
    nz, ny, nx = f.shape
    kx = 2 * np.pi * np.fft.fftfreq(nx, d=dx)
    ky = 2 * np.pi * np.fft.fftfreq(ny, d=dy)
    kz = 2 * np.pi * np.fft.fftfreq(nz, d=dz)
    kz, ky, kx = np.meshgrid(kz, ky, kx, indexing='ij')
    lap_k = -(kx**2 + ky**2 + kz**2)
    lap_k[0,0,0] = 1  # avoid divide-by-zero
    f_hat = fftw.fftn(f)
    phi_hat = f_hat / lap_k
    phi_hat[0,0,0] = 0  # zero mean mode
    phi = fftw.ifftn(phi_hat).real
    return phi

def LaplacianSpectral(phi, dx=1.0, dy=1.0, dz=1.0):
    nz, ny, nx = phi.shape
    kx = 2 * np.pi * np.fft.fftfreq(nx, d=dx)
    ky = 2 * np.pi * np.fft.fftfreq(ny, d=dy)
    kz = 2 * np.pi * np.fft.fftfreq(nz, d=dz)
    kz, ky, kx = np.meshgrid(kz, ky, kx, indexing='ij')
    lap_k = -(kx**2 + ky**2 + kz**2)
    phi_hat = fftw.fftn(phi)
    lap_phi_hat = lap_k * phi_hat
    lap_phi = fftw.ifftn(lap_phi_hat).real
    return lap_phi

# Grid setup
nx, ny, nz = 64, 64, 64
Lx, Ly, Lz = 16.0, 16.0, 16.0
x = np.linspace(-Lx/2, Lx/2, nx)
y = np.linspace(-Ly/2, Ly/2, ny)
z = np.linspace(-Lz/2, Lz/2, nz)
dx = x[1] - x[0]
dy = y[1] - y[0]
dz = z[1] - z[0]
Z, Y, X = np.meshgrid(z, y, x, indexing='ij')

# --- Periodic test function ---
nx_mode, ny_mode, nz_mode = 2, 3, 1  # integer wavenumbers
kx = 2 * np.pi * nx_mode / Lx
ky = 2 * np.pi * ny_mode / Ly
kz = 2 * np.pi * nz_mode / Lz

phi_true = np.sin(kx * X) * np.cos(ky * Y) * np.sin(kz * Z)
f = -(kx**2 + ky**2 + kz**2) * phi_true  # Analytical Laplacian

# Invert Laplacian to get phi
phi = InvertLaplacian3DPeriodic(f, dx=dx, dy=dy, dz=dz)

# Compute Laplacian(phi) numerically
lap_phi = LaplacianSpectral(phi, dx=dx, dy=dy, dz=dz)

# Errors
error_phi = phi - phi_true
rmse_phi = np.sqrt(np.mean(error_phi**2))
print(f"RMSE between recovered phi and true phi: {rmse_phi:.3e}")

error_f = lap_phi - f
rmse_f = np.sqrt(np.mean(error_f**2))
print(f"RMSE between Laplacian(phi) and f: {rmse_f:.3e}")

# Plot slice at middle z
slice_idx = nz // 2

plt.figure(figsize=(20,4))

plt.subplot(1,4,1)
plt.title('Source function f')
plt.imshow(f[slice_idx,:,:], origin='lower', extent=[x[0], x[-1], y[0], y[-1]])
plt.colorbar()

plt.subplot(1,4,2)
plt.title('phi')
plt.imshow(phi[slice_idx,:,:], origin='lower', extent=[x[0], x[-1], y[0], y[-1]])
plt.colorbar()

plt.subplot(1,4,3)
plt.title('Laplacian(phi)')
plt.imshow(lap_phi[slice_idx,:,:], origin='lower', extent=[x[0], x[-1], y[0], y[-1]])
plt.colorbar()

plt.subplot(1,4,4)
plt.title('Error: phi - phi_true')
plt.imshow(error_phi[slice_idx,:,:], origin='lower', extent=[x[0], x[-1], y[0], y[-1]], cmap='bwr')
plt.colorbar()

plt.tight_layout()
plt.show()


In [ ]:
# Grid setup
nx, ny, nz = 64, 64, 64
Lx, Ly, Lz = 16.0, 16.0, 16.0
x = np.linspace(0, Lx, nx, endpoint=False)
y = np.linspace(0, Ly, ny, endpoint=False)
z = np.linspace(0, Lz, nz, endpoint=False)
dx = x[1] - x[0]
dy = y[1] - y[0]
dz = z[1] - z[0]
Z, Y, X = np.meshgrid(z, y, x, indexing='ij')

# --- Periodic test function ---
nx_mode, ny_mode, nz_mode = 2, 3, 1
kx = 2 * np.pi * nx_mode / Lx
ky = 2 * np.pi * ny_mode / Ly
kz = 2 * np.pi * nz_mode / Lz

phi_true = np.sin(kx * X) * np.cos(ky * Y) * np.sin(kz * Z)
f = -(kx**2 + ky**2 + kz**2) * phi_true

# Invert Laplacian to get phi
phi = InvertLaplacian3DPeriodic(f, dx=dx, dy=dy, dz=dz)

# Compute Laplacian(phi) numerically
lap_phi = LaplacianSpectral(phi, dx=dx, dy=dy, dz=dz)

# Errors
error_phi = phi - phi_true
rmse_phi = np.sqrt(np.mean(error_phi**2))
print(f"RMSE between recovered phi and true phi: {rmse_phi:.3e}")

error_f = lap_phi - f
rmse_f = np.sqrt(np.mean(error_f**2))
print(f"RMSE between Laplacian(phi) and f: {rmse_f:.3e}")

# Plot slice at middle z
slice_idx = nz // 2

plt.figure(figsize=(20,4))

plt.subplot(1,4,1)
plt.title('Source function f')
plt.imshow(f[slice_idx,:,:], origin='lower', extent=[x[0], x[-1], y[0], y[-1]])
plt.colorbar()

plt.subplot(1,4,2)
plt.title('phi')
plt.imshow(phi[slice_idx,:,:], origin='lower', extent=[x[0], x[-1], y[0], y[-1]])
plt.colorbar()

plt.subplot(1,4,3)
plt.title('Laplacian(phi)')
plt.imshow(lap_phi[slice_idx,:,:], origin='lower', extent=[x[0], x[-1], y[0], y[-1]])
plt.colorbar()

plt.subplot(1,4,4)
plt.title('Error: phi - phi_true')
plt.imshow(error_phi[slice_idx,:,:], origin='lower', extent=[x[0], x[-1], y[0], y[-1]], cmap='bwr')
plt.colorbar()

plt.tight_layout()
plt.show()


In [ ]:
#SOLVING FOR BUOYANCY PRESSURE 

In [ ]:
# import dask.array as da
# from dask.diagnostics import ProgressBar

# # Create a dask array by chunking the xarray variable
# rho = data1['rho'].chunk({'time': 10, 'zh': 10, 'yh': 50, 'xh': 50})
# rho_array = rho.data  # this is the pure dask array

# with ProgressBar():
#     rho_mean = rho.mean(axis=None).compute()

# print("Total mean of rho:", rho_mean)

# import dask.array as da
# from dask.diagnostics import ProgressBar

# # Chunk the xarray variable
# rho = data1['rho'].chunk({'time': 10, 'zh': 10, 'yh': 50, 'xh': 50})
# rho_array = rho.data

# # Use the threaded scheduler for potentially faster execution
# with ProgressBar():
#     rho_mean = rho.mean().compute(scheduler='threads')

# print("Total mean of rho:", rho_mean)
# rho = data1['rho'].chunk({'time': 10, 'zh': 10, 'yh': 50, 'xh': 50})
# rho_array = rho.data  # this is the pure dask array

# with ProgressBar():
#     rho_mean = rho.mean(axis=None).compute()

# print("Total mean of rho:", rho_mean)


In [ ]:
# #VARIABLES
# rho_t=data1['rho'].isel(time=100).data
# rho0=np.mean(rho_t)
# B=data1['buoyancy'].isel(time=100).data
# rho0B=rho0*B

# #RHS
# f=Ddz_3DStretch(rho0B,data1)

In [ ]:
# InvertLaplacian3DPeriodic(f,dx=1000,dy=1000)